In [54]:
import Config as conf
import Lockin
import FileInfo
import matplotlib.pyplot as plt
%matplotlib qt
import numpy as np
import math
import os

QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-nishi_gaku'


In [61]:
StartNo = FileInfo.GetMaxFileNumber() + 1
StopNo  = conf.NumOfDataAcquisition

file_list = os.listdir(conf.DataPath)
data_list = []

for i in range(len(file_list)):
    if ".bin" == os.path.splitext(file_list[i])[1]:
        data_list.append(os.path.join(conf.DataPath, file_list[i]))

for i in range(len(data_list)):
    BinaryFileName = data_list[i]
    V, Time = Lockin.Lockin(BinaryFileName)
    #print("read %d th data" %i)
    if i == 0:
        #plt.plot(Time, V, ".")
        #plt.show()
        V_mean = np.array([0.0 for i in range(len(V))])
    V_mean += V

[122, 351, 206, 96, 190, 183, 176, 77, 125, 68, 299, 334, 129, 191, 5, 133, 55, 8, 124, 88, 34, 273, 84, 106, 309, 62, 278, 222, 180, 168, 132, 304, 338, 164, 155, 270, 220, 175, 60, 279, 1, 179, 79, 356, 250, 263, 85, 24, 193, 292, 297, 343, 245, 26, 45, 111, 333, 313, 117, 319, 244, 25, 196, 339, 136, 157, 205, 46, 150, 142, 285, 120, 325, 161, 71, 172, 241, 295, 290, 328, 314, 226, 255, 266, 332, 92, 323, 342, 268, 354, 347, 90, 335, 89, 208, 91, 126, 10, 317, 36, 178, 69, 53, 43, 54, 239, 30, 254, 329, 302, 40, 158, 42, 252, 112, 243, 340, 181, 324, 186, 19, 360, 114, 233, 312, 224, 73, 102, 200, 289, 337, 214, 108, 94, 232, 240, 348, 159, 39, 321, 63, 4, 311, 353, 271, 140, 2, 358, 282, 83, 330, 103, 284, 134, 260, 350, 184, 98, 269, 265, 149, 231, 141, 47, 66, 104, 357, 23, 105, 6, 229, 58, 22, 27, 216, 218, 52, 81, 251, 20, 315, 286, 320, 107, 86, 14, 95, 238, 41, 213, 267, 116, 174, 217, 152, 72, 51, 113, 207, 167, 118, 275, 18, 173, 80, 38, 87, 151, 236, 230, 153, 219, 203, 30

In [68]:
def FindFreqDeltaLockin(StartF, EndF, Naverage, Time, V_mean):
    FreqList = []
    deltaList = []
    LockinList = []
    
    deltaRange = np.linspace(0., 2*math.pi, 100)
    #deltaRange = np.linspace(0.9*math.pi, 1.1*math.pi, 50)
    for f0 in range(StartF, EndF, 1):
        for delta in deltaRange:
            LockinValue = 0
            for i in range(Naverage):
                LockinValue += V_mean[i]*math.cos(2*math.pi*f0*Time[i]+delta)
                if i == Naverage-1:
                    FreqList.append(f0)
                    deltaList.append(delta)
                    LockinList.append(LockinValue)
                    
    return FreqList, deltaList, LockinList

def FindFreqLockin(StartF, EndF, Naverage, Time, V_mean):
    FreqList = []
    LockinList = []
    
    for f0 in range(StartF, EndF, 1):
        LockinValue = 0
        for i in range(Naverage):
            LockinValue += V_mean[i]*math.cos(2*math.pi*f0*Time[i]+math.pi)
            #LockinValue += V_mean[i]*math.cos(2*math.pi*f0*Time[i])
            if i == Naverage-1:
                FreqList.append(f0)
                LockinList.append(LockinValue)
                    
    return FreqList, LockinList

def FindDeltaLockin(f0, Naverage, Time, V_mean):
    FreqList = []
    deltaList = []
    LockinList = []
    
    deltaRange = np.linspace(0., 2*math.pi, 50)
    for delta in deltaRange:
        LockinValue = 0
        for i in range(Naverage):
            LockinValue += V_mean[i]*math.cos(2*math.pi*f0*Time[i]+delta)
            #LockinValue += V_mean[i]*math.cos(2*math.pi*f0*Time[i])
            if i == Naverage-1:
                deltaList.append(delta)
                LockinList.append(LockinValue)
                    
    return deltaList, LockinList

In [69]:
FreqList, deltaList, LockinList = FindFreqDeltaLockin(17000, 19000, 1000, Time, V_mean)
"""
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(FreqList, deltaList, LockinList)
plt.show()
"""

"\nfig = plt.figure()\nax = fig.add_subplot(projection='3d')\nax.scatter(FreqList, deltaList, LockinList)\nplt.show()\n"

In [70]:
import pandas as pd
df = pd.DataFrame({"freq": FreqList, "delta":deltaList, "Lockin": LockinList})

In [71]:
df_s = df.sort_values(by = "Lockin", ascending=False)
df_s[:10]

,freq,delta,Lockin
127085,18270,5.394654,1002.336058
127582,18275,5.204254,1002.320115
126588,18265,5.585054,1002.319007
126290,18262,5.711987,1002.286749
126787,18267,5.521587,1002.282337
127880,18278,5.077321,1002.271448
128079,18280,5.013855,1002.271195
126091,18260,5.775453,1002.268947
127383,18273,5.267721,1002.265921
125793,18257,5.902386,1002.258147


In [48]:
(3.205707-math.pi)/math.pi

0.02040823030858104